In [1]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import scipy as sp
import datetime
import math
import operator
import re
import os
import os.path
import urllib, json
import csv
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from operator import itemgetter
from scipy.spatial import KDTree
from numpy import linalg as LA
from calendar import monthrange
from sklearn import metrics
import tarfile
import dask
import glob
#from deepmoji.sentence_tokenizer import SentenceTokenizer
#from deepmoji.model_def import deepmoji_emojis
#from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
#from deepmoji.model_def import deepmoji_transfer
#from deepmoji.global_variables import PRETRAINED_PATH
#from deepmoji.finetuning import (
#    load_benchmark,
#    finetune)

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask.multiprocessing
from dask import compute, delayed

import time
%matplotlib inline

/home/s132732/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
YEAR="2014"

In [3]:
listing = glob.glob(os.path.join('./', '2014_*.out'))

In [4]:
files=sorted(listing, key=lambda name: int(name.split('_')[1]))

In [5]:
df_from_each_file = (pd.read_csv(f,
                 sep=",", #or delim_whitespace=True, #separator is whitespace
                 header=None, #no header
                 names=['JOY','FEAR','ANGER','SADNESS','DISGUST','SHAME','GUILT'] #set columns names joy, fear, anger, sadness, disgust, shame, guilt
                 )  for f in files)
emoji_df   = pd.concat(df_from_each_file, ignore_index=True)


In [6]:
emoji_df.shape

(79711096, 7)

In [7]:
ill_df=pd.read_csv('data/US_2014_integrated.csv',encoding='utf-8',index_col=0) 
YEAR=2014
#ill_df['timestamp']=pd.to_datetime(ill_df['timestamp'], errors='coerce')
ill_df.shape

/home/s132732/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(79711096, 15)

In [8]:
ill_df=ill_df.reset_index(drop=True)
ill_df.head()

,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,HUM,CC,WIND,POP,month
0,1,24.547203,9,-81.790858,2014-01-01,"[im, at, casa, marina, pier, key, west, fl, ht...",True,25.75,21.879999,0.0,83.75,87.0,7.074967,833.0373,1
1,1,24.547391,18,-81.787761,2014-01-01,"[looks, awesome, very, excited, to, be, here, ...",True,25.75,21.879999,0.0,83.75,87.0,7.074967,833.0373,1
2,1,24.547487,15,-81.794067,2014-01-01,"[key, west, fl, , #last, #days, #of, #christma...",True,25.75,21.879999,0.0,83.75,87.0,7.074967,1560.5700,1
3,1,24.547887,6,-81.802588,2014-01-01,"[happy, new, year, world, #2k14, ]",True,25.75,21.879999,0.0,83.75,87.0,7.074967,1757.6073,1
4,1,24.548268,5,-81.794850,2014-01-01,"[@doubledena, dena, marie, what, happened]",True,25.75,21.879999,0.0,83.75,87.0,7.074967,1560.5700,1


In [9]:
twe_emo_df = pd.concat([ill_df, emoji_df],axis=1)
twe_emo_df.head()

,count,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,...,WIND,POP,month,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT
0,1,24.547203,9,-81.790858,2014-01-01,"[im, at, casa, marina, pier, key, west, fl, ht...",True,25.75,21.879999,0.0,...,7.074967,833.0373,1,0.488879,0.125108,0.077746,0.042987,0.065479,0.086794,0.113007
1,1,24.547391,18,-81.787761,2014-01-01,"[looks, awesome, very, excited, to, be, here, ...",True,25.75,21.879999,0.0,...,7.074967,833.0373,1,0.813676,0.031647,0.040933,0.012899,0.019053,0.035037,0.046756
2,1,24.547487,15,-81.794067,2014-01-01,"[key, west, fl, , #last, #days, #of, #christma...",True,25.75,21.879999,0.0,...,7.074967,1560.5700,1,0.438682,0.119623,0.095877,0.057260,0.088380,0.106697,0.093480
3,1,24.547887,6,-81.802588,2014-01-01,"[happy, new, year, world, #2k14, ]",True,25.75,21.879999,0.0,...,7.074967,1757.6073,1,0.542445,0.077370,0.084879,0.079179,0.047340,0.099174,0.069612
4,1,24.548268,5,-81.794850,2014-01-01,"[@doubledena, dena, marie, what, happened]",True,25.75,21.879999,0.0,...,7.074967,1560.5700,1,0.093814,0.156439,0.158342,0.120227,0.154013,0.160930,0.156236


In [20]:

pixelwidth=0.0416666666667 #PRISM cell size (800m)
width_50=pixelwidth*12.5
width_25=pixelwidth*6.25
### BOunding box for Illinois
westlon=-124.7844079
southlat=24.7433195
eastlon=-66.9513812
northlat=49.3457868

resol_lon= int(round(abs(westlon-eastlon)/width_50))
resol_lat= int(round(abs(southlat-northlat)/width_50))

n_lat, n_lon=(resol_lat, resol_lon)
lat_bins = np.linspace(southlat, northlat, n_lat)
lon_bins = np.linspace(westlon, eastlon, n_lon)

lat_labels=np.arange(resol_lat-1)
lon_labels=np.arange(resol_lon-1)

### Bin all observations
twe_emo_df = twe_emo_df.assign(
    LAT_cut=pd.cut(twe_emo_df.LAT,lat_bins,labels=False),
    LON_cut=pd.cut(twe_emo_df.LON,lon_bins,labels=False)
)

twe_emo_df=twe_emo_df.rename(index=str, columns={"count": "count_daily"})
twe_emo_df['timestamp']=pd.to_datetime(twe_emo_df['timestamp'], errors='coerce')
### Categorize on Urban or NOt
twe_emo_df.dropna(axis=0, how='any',inplace=True)
ill_urban_df=twe_emo_df[twe_emo_df['URBAN']==True]
ill_rural_df=twe_emo_df[twe_emo_df['URBAN']==False]


print "Spatial Bin on LAT/LON with resolution (n_lat, n_lon):",(resol_lat, resol_lon)
#print lat_bins
#print lon_bins
twe_emo_df.head()

Spatial Bin on LAT/LON with resolution (n_lat, n_lon): (47, 111)


,count_daily,LAT,length,LON,timestamp,txt,URBAN,TMAX,TMIN,PRCP,...,month,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT,LAT_cut,LON_cut
154,1,24.768614,4,-80.909899,2014-01-01,"[and, time, ran, out]",True,26.35,20.99,1.22,...,1,0.039201,0.225378,0.187586,0.243164,0.053000,0.079247,0.172424,0,83
155,1,24.768718,6,-80.909966,2014-01-01,"[what, the, shit, you, little, shit]",True,26.35,20.99,1.22,...,1,0.064869,0.096623,0.209226,0.119475,0.235421,0.135385,0.139001,0,83
156,1,24.768724,8,-80.909950,2014-01-01,"[ayy, i, wanna, go, to, the, bangerz, concert]",True,26.35,20.99,1.22,...,1,0.299170,0.154603,0.090451,0.124990,0.111823,0.110532,0.108432,0,83
157,1,24.769033,10,-80.910064,2014-01-01,"[@ohteenposts, i, love, how, the, disney, star...",True,26.35,20.99,1.22,...,1,0.361341,0.131721,0.105222,0.117984,0.087015,0.106627,0.090091,0,83
158,1,24.840976,9,-80.788750,2014-01-01,"[@anneliheed, oh, its, easy, i, can, even, tea...",False,25.52,21.48,1.05,...,1,0.163696,0.111318,0.125569,0.085107,0.116952,0.209692,0.187665,0,83


In [21]:
ill_urban_monthlygrouped_df=ill_urban_df.groupby([ill_urban_df.month,ill_urban_df.LAT_cut,ill_urban_df.LON_cut])['count_daily'].sum().to_frame().reset_index()
ill_rural_monthlygrouped_df=ill_rural_df.groupby([ill_rural_df.month,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_urban_monthlygrouped_df=ill_urban_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_urban"})
ill_rural_monthlygrouped_df=ill_rural_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_rural"})

ill_urban_monthlygrouped_df['monthly_cell_daily_count'] = ill_urban_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_urban'] / monthrange(YEAR,int(row.month))[1], axis=1)
ill_rural_monthlygrouped_df['monthly_cell_daily_count'] = ill_rural_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_rural'] / monthrange(YEAR,int(row.month))[1], axis=1)

ill_merge_monthlygrouped_df=pd.merge(ill_urban_monthlygrouped_df, ill_rural_monthlygrouped_df, how='outer', on=['month','LAT_cut','LON_cut'], sort=True,
         suffixes=('_urban', '_rural'))

ill_merge_monthlygrouped_df.fillna(0).head() ##MONTHLY DAILY CELL COUNT

,month,LAT_cut,LON_cut,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,1,0,83,118.0,3.806452,35.0,1.129032
1,1,0,84,410.0,13.225806,49.0,1.580645
2,1,1,83,0.0,0.000000,120.0,3.870968
3,1,1,84,60650.0,1956.451613,1679.0,54.161290
4,1,2,49,321.0,10.354839,52.0,1.677419


In [22]:
ill_urban_dailygrouped_df=ill_urban_df.groupby([ill_urban_df.timestamp,ill_urban_df.LAT_cut,ill_urban_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
#ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_merge_dailygrouped_df=pd.merge(ill_urban_dailygrouped_df, ill_rural_dailygrouped_df, how='outer', on=['timestamp','LAT_cut','LON_cut','month'], sort=True, suffixes=('_urban', '_rural'))

ill_merge_daily_df=pd.merge(ill_merge_dailygrouped_df,ill_merge_monthlygrouped_df, how="left", on=['LAT_cut','LON_cut','month'])

ill_merge_daily_df=ill_merge_daily_df
#f, ax = plt.subplots(1, 1, sharey=True)
#f.set_size_inches(18.5, 10.5, forward=True)

#test=ill_merge_daily_df[ill_merge_daily_df['timestamp']=='2016-02-21']
#plt.scatter(test['LON_cut'], test['LAT_cut'], s=700, c=test['PRCP_rural'], cmap="Reds", alpha=0.5,edgecolors=None,linewidths=0)
ill_merge_daily_df['timestamp']=ill_merge_daily_df['timestamp'].dt.strftime("%Y-%m-%d")
ill_merge_daily_df.head()

,timestamp,LAT_cut,LON_cut,TMIN_urban,CC_urban,month,count_daily_urban,SADNESS_urban,GUILT_urban,DISGUST_urban,...,TMAX_rural,SHAME_rural,HUM_rural,ANGER_rural,PRCP_rural,WIND_rural,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,2014-01-01,0,83,21.703846,93.699951,1,13.0,0.090945,0.115088,0.154778,...,25.665000,0.141052,84.75,0.109844,1.850000,4.632286,118.0,3.806452,35.0,1.129032
1,2014-01-01,0,84,21.939131,93.699951,1,46.0,0.115548,0.125157,0.134144,...,26.310000,0.161021,84.75,0.157531,2.662000,4.632286,410.0,13.225806,49.0,1.580645
2,2014-01-01,1,83,NaN,NaN,1,NaN,NaN,NaN,NaN,...,25.310000,0.160954,84.75,0.162110,0.080000,4.632286,NaN,NaN,120.0,3.870968
3,2014-01-01,1,84,20.794177,93.699951,1,2406.0,0.099693,0.131837,0.135068,...,24.840417,0.132649,84.75,0.123796,0.732778,4.632286,60650.0,1956.451613,1679.0,54.161290
4,2014-01-01,2,49,5.284706,98.899902,1,17.0,0.090219,0.131892,0.141602,...,8.155000,0.145971,92.00,0.151351,18.830000,0.599084,321.0,10.354839,52.0,1.677419


In [24]:
ill_count_daily_df=pd.read_csv('data/US_2014_50KMbinned.csv',encoding='utf-8',index_col=0)

In [25]:
print "Emo shape ", ill_merge_daily_df.shape
print "Count shape ", ill_count_daily_df.shape

Emo shape  (763475, 36)
Count shape  (763475, 32)


In [26]:
ill_count_daily_df.head()

,Unnamed: 0.1,timestamp,LAT_cut,LON_cut,count_daily_urban,TMIN_urban,CC_urban,HUM_urban,TMAX_urban,PRCP_urban,...,monthly_cell_daily_count_rural,TMAX_HIS_mean,TMIN_HIS_mean,PRCP_HIS_mean,HUM_HIS_mean,WIND_HIS_mean,CC_HIS_mean,ncount_daily_urban,ncount_daily_rural,cell_id
0,0,2014-01-01,0.0,83.0,13.0,21.703846,93.699951,84.750000,25.973846,1.965385,...,1.129032,NaN,NaN,NaN,80.268669,7.187416,57.916634,9.193548,2.870968,"(0.0, 83.0)"
1,1,2014-01-01,0.0,84.0,46.0,21.939131,93.699951,84.750000,26.173913,4.588261,...,1.580645,NaN,NaN,NaN,80.268669,7.187416,57.916634,32.774194,3.419355,"(0.0, 84.0)"
2,2,2014-01-01,1.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.870968,24.817665,16.374998,1.972334,80.268669,7.187416,57.916634,NaN,-1.870968,"(1.0, 83.0)"
3,3,2014-01-01,1.0,84.0,2406.0,20.794177,93.699951,84.750000,24.772135,0.930999,...,54.161290,25.745331,15.748001,0.731333,80.268669,7.187416,57.916634,449.548387,25.838710,"(1.0, 84.0)"
4,4,2014-01-01,2.0,49.0,17.0,5.284706,98.899902,84.514706,8.027059,19.777647,...,1.677419,NaN,NaN,NaN,70.404663,4.145273,55.726627,6.645161,0.322581,"(2.0, 49.0)"


In [27]:
cols_to_use = ill_merge_daily_df.columns.difference(ill_count_daily_df.columns)
cols_to_use = cols_to_use.tolist()+ ['timestamp', 'LAT_cut','LON_cut']
print cols_to_use

['ANGER_rural', 'ANGER_urban', 'DISGUST_rural', 'DISGUST_urban', 'FEAR_rural', 'FEAR_urban', 'GUILT_rural', 'GUILT_urban', 'JOY_rural', 'JOY_urban', 'SADNESS_rural', 'SADNESS_urban', 'SHAME_rural', 'SHAME_urban', 'timestamp', 'LAT_cut', 'LON_cut']


In [28]:
ill_emo_count_daily_df = pd.merge(ill_count_daily_df.drop(['Unnamed: 0.1'], axis=1), ill_merge_daily_df[cols_to_use], how='inner', on=['timestamp', 'LAT_cut','LON_cut'],validate="one_to_one")

In [29]:
print "Merge Shape ",ill_emo_count_daily_df.shape

Merge Shape  (763475, 45)


In [30]:
ill_emo_count_daily_df.to_csv('data/US_2014_EMO_50KMbinned.csv',encoding='utf-8')